In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek2 import env
from ronek2 import const
from ronek2 import utils
from ronek2 import backend as bkd
from ronek2.systems import BoxAd
from ronek2.roms import CoBRAS

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True
)
system.compute_c_mat(max_mom=2)

In [6]:
rho = 1e-2
T = 3e4
Te = 3e2
mu_quad = {
  "x": np.array([rho, T, Te]).reshape(1,-1),
  "w": np.ones((1,1))
}

In [7]:
cobras = CoBRAS(
  system=system,
  tgrid={"start": 1e-12, "stop": 1e-3, "num": 25},
  mu_quad=mu_quad,
  path_to_saving="./",
  saving=True
)

In [ ]:
cobras(
  nb_meas=4,
  xnot=[-2,-1],
  modes=True,
  pod=True
)

In [ ]:
stop

In [ ]:
X, Y = cobras(
  nb_meas=3,
  xnot=[-2,-1],
  modes=False,
  pod=True
)

In [ ]:
X.shape, Y.shape

In [ ]:
Y[:,500]

In [12]:
Ux, sx, Vhx = sp.linalg.svd(X)
Uy, sy, Vhy = sp.linalg.svd(Y)

In [ ]:
sx.max()*sy.min(), sx.min()*sy.max()

In [14]:
# Compute the initial solution for the system
y0, rho = system.equil.get_init_sol(mu_quad["x"].squeeze(), noise=False, sigma=1e-1)
# Determine the smallest time scale for resolving system dynamics
tmin = system.compute_lin_tscale(y0, rho, smallest=True)
# Generate a time quadrature grid and associated weights
t, w_t = cobras.get_tquad(tmin)

In [15]:
y = system.solve_fom(t, y0, rho, linear=False)[0]

In [16]:
w_true = copy.deepcopy(y[:system.mix.nb_comp])
n_true = system.mix.get_n(bkd.to_torch(w_true)).numpy()
T_true = copy.deepcopy(y[system.mix.nb_comp:])
T_true[-1] = system.mix.get_Te(pe=T_true[-1], ne=n_true[-1])

In [17]:
ylin = system.solve_fom(t, y0, rho, linear=True)[0]
# ylin = system.solve_fom(t[28:]-t[28], y[:,28], rho, linear=True)[0]

In [ ]:
(t[28:]-t[28])[11]

In [ ]:
ylin.shape

In [20]:
err = utils.absolute_percentage_error(y, ylin, eps=0.0)

In [21]:
w_pred = copy.deepcopy(ylin[:system.mix.nb_comp])
n_pred = system.mix.get_n(bkd.to_torch(w_pred)).numpy()
T_pred = copy.deepcopy(ylin[system.mix.nb_comp:])
T_pred[-1] = system.mix.get_Te(pe=T_pred[-1], ne=n_pred[-1])

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.loglog(t[1:-10], T_true[i][1:-10])
  plt.loglog(t[1:-10], T_pred[i][1:-10], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in system.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_true[i] - n_pred[i])/n_true[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

In [ ]:
# Compute the error between nonlinear and linear solutions
err = utils.absolute_percentage_error(y, ylin, eps=0.0)
# Average the error across the state dimension
err = np.mean(err, axis=0)
# Find the last index where the error is within the threshold
idx = np.argmin(np.abs(err - 25.0))
print(idx, t[idx])

In [ ]:
plt.plot()

In [ ]:
y0 = np.zeros(system.nb_eqs)
y0[:system.mix.nb_comp] = rho_true[:,0] / rho
y0[-2] = Th0
y0[-1] = system.mix.get_pe(Te=Te0, ne=n_true[-1,0])
y0

In [ ]:
y, runtime = system.solve_fom(t, y0, rho, linear=False)
runtime

In [13]:
w_pred = copy.deepcopy(y[:system.mix.nb_comp])
n_pred = system.mix.get_n(bkd.to_torch(w_pred)).numpy()
T_pred = copy.deepcopy(y[system.mix.nb_comp:])
T_pred[-1] = system.mix.get_Te(pe=T_pred[-1], ne=n_pred[-1])

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.semilogx(t[1:], T_true[i][1:])
  plt.semilogx(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in system.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_true[i] - n_pred[i])/n_true[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

Linear system

In [ ]:
ylin, runtime = system.solve_fom(t, y0, rho, linear=True)
runtime

In [17]:
w_lin = copy.deepcopy(ylin[:system.mix.nb_comp])
n_lin = system.mix.get_n(bkd.to_torch(w_lin)).numpy()
T_lin = copy.deepcopy(ylin[system.mix.nb_comp:])
T_lin[-1] = system.mix.get_Te(pe=T_lin[-1], ne=n_lin[-1])

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_pred[i] - T_lin[i])/T_pred[i]))
  err.append(ierr)
  plt.semilogx(t[1:-200], T_pred[i][1:-200])
  plt.semilogx(t[1:-200], T_lin[i][1:-200], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in system.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_pred[i] - n_lin[i])/n_pred[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_pred[i][1:])
    plt.loglog(t[1:], n_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

Sample time points

In [20]:
y_interp = sp.interpolate.interp1d(t, y, kind="linear", axis=-1)

In [21]:
tvec = np.array([0.0] + np.geomspace(1e-12, 1e-4, 100).tolist())
tsamples = np.geomspace(1e-13, 1e-4, 10)

In [ ]:
for i, ti in enumerate(tsamples):
  
  print(f"Time {i+1}: {ti} s")
  print("="*40)

  yref = y_interp(tvec+ti)
  ylin, _ = system.solve_fom(tvec, yref[:,0], rho, linear=True)

  tau1 = system.compute_lin_tlim(tvec, yref, rho, use_eig=True, err_max=25.0)
  tau2 = system.compute_lin_tlim(tvec, yref, rho, use_eig=False, err_max=25.0)
  print(tau1, tau2)

  for s in system.mix.species.values():
    for i in s.indices:
      plt.loglog(tvec[1:], yref[i][1:])
      plt.loglog(tvec[1:], ylin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
    plt.axvline(x=tau1[1], color='r', linestyle='--')
    plt.axvline(x=tau2[1], color='g', linestyle='--')
    plt.show()
    plt.close()